In [70]:
from pymilvus import connections, utility, MilvusException, FieldSchema, CollectionSchema, DataType, Collection, loading_progress
connections.connect(host="localhost", port="19530", db_name='default')

In [71]:
fields = [
    FieldSchema(name="Data_Id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="Dataset_File", dtype=DataType.VARCHAR,max_length=255),
    FieldSchema(name="Collected", dtype=DataType.VARCHAR,max_length=100),
    FieldSchema(name="Variable", dtype=DataType.VARCHAR,max_length=100),
    FieldSchema(name="Label", dtype=DataType.VARCHAR,max_length=255),
    FieldSchema(name="Description", dtype=DataType.FLOAT_VECTOR, dim=1024),
]

collection_schema = CollectionSchema(fields, description="Example_collection")

doc_data = Collection("Milvus_Test", collection_schema)


In [72]:
#Index on vector
def Create_Index(collection_Name: str, vector_name: str, index_par: dict):
    try:
        collection = Collection(collection_Name)
        collection.create_index(field_name = vector_name, index_params = index_par)
    except Exception as e:
        print(e)
    else:
        print("Index created")

index_params = {
    "index_type" : "IVF_FLAT",
    "metric_type" : "L2",
    "params" : {"nlist":128}  ,
}

Create_Index("Milvus_Test", "Description", index_params) 


Index created


In [56]:
#insert function 
#NOT TEST YET
def Insert_vector(id_list: list, data_file_list: list, collected_list: list, variable_list: list, label_list: list, embeddings_Description, Collection):
    try:
        collection = Collection
 
        for id_item, data_file_item, collected_item, variable_item, label_item, embeddings_Description in zip(id_list, data_file_list, collected_list, variable_list, label_list, embeddings_Description):
            entities = [[id_item], [data_file_item], [collected_item], [variable_item], [label_item], [embeddings_Description]]
            collection.insert(entities)
            print(f"编号{id_item}插入完毕")
        collection.flush()
 
    except Exception as e:
        print(e)

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

Index(['Dataset File', 'Collected', 'Variable', 'Label', 'Description'], dtype='object')
<class 'str'>
Have you ever had a Pap smear (a cancer check done during a female exam)?


Inference Embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 448/448 [09:04<00:00,  1.22s/it]
RPC error: [batch_insert], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 5)>, <Time:{'RPC start': '2024-03-21 23:02:50.108098', 'RPC error': '2024-03-21 23:02:50.108098'}>


<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 5)>
插入数据用时为: 0.0009999275207519531秒, 平均一条的插入时间为: 1.3953775059335098e-07


In [75]:
import time
import pandas as pd
import json

from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_ef = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu', # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
)

Collection_Name = 'Milvus_Test'
collection = Collection(Collection_Name)

file_path = 'data_1.csv'
df = pd.read_csv(file_path)
print(df.columns)

id_list = df.index.tolist()
dataset_file_data = df["Dataset File"].tolist()
collected_data = df.Collected.tolist()
variable_data = df.Variable.tolist()
label_data = df.Label.tolist()

df['Description'] = df['Description'].fillna('no description')
description_data = df['Description'].tolist()

print(type(description_data[100]))
print(description_data[100])
#embedding
Description_embedding = bge_m3_ef.encode_documents(description_data)["dense"]

print(type(Description_embedding))
#######################################################################################

try:
    filename = 'embedding_data.json'

    with open(filename, 'w') as json_file:
        json.dump(Description_embedding, json_file, ensure_ascii = False) 
        
    dimension = len(Description_embedding[0])
    
    print(f'The dimension of the vectors is {dimension}')
except Exception as e:
    print(f"writing error: {e}")
#######################################################################################
# d_embed = Description_embedding.tolist()
start_time = time.time()
try:
    insert_data = [id_list, dataset_file_data, collected_data, variable_data, label_data, Description_embedding]
    mr = collection.insert(insert_data)
    print(f"成功插入数据，影响的行数：{len(mr.primary_keys)}")
except Exception as e:
    print(f"插入数据时发生错误：{e}")
end_time = time.time()
print(f"插入数据用时为: {end_time - start_time}秒, 平均一条的插入时间为: {(end_time - start_time)/len(id_list)}")
print(f"总共有: {len(id_list)}条数据")

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

Index(['Dataset File', 'Collected', 'Variable', 'Label', 'Description'], dtype='object')
<class 'str'>
Have you ever had a Pap smear (a cancer check done during a female exam)?


Inference Embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 448/448 [07:43<00:00,  1.03s/it]


<class 'list'>


RPC error: [batch_insert], <ParamError: (code=1, message=expect string input, got: <class 'float'>)>, <Time:{'RPC start': '2024-03-22 03:13:39.796348', 'RPC error': '2024-03-22 03:13:39.805326'}>


The dimension of the vectors is 1024
插入数据时发生错误：<ParamError: (code=1, message=expect string input, got: <class 'float'>)>
插入数据用时为: 0.010972261428833008秒, 平均一条的插入时间为: 1.5311556557121137e-06
总共有: 7166条数据
